# lightCnnGPU_MFCC
urban sound 예제에서 쓰인 feature extraction을 적용시켜 보자 

In [191]:
import os
import numpy as np
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy import signal
from glob import glob
import re
import pandas as pd
import gc
from scipy.io import wavfile

from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import keras
import time
import datetime

from keras.models import Sequential 
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import h5py

# import keras necessary classes
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
import numpy as np
import pandas as pd

import numpy as np
import random
import itertools
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt


import matplotlib.pyplot as plt


import numpy as np
import random
import itertools
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt


#import glob
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
%matplotlib notebook
plt.style.use('ggplot')

In [192]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1392330768640239658
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 202047488
locality {
  bus_id: 1
}
incarnation: 8074321606245386709
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0"
]


In [193]:
parent_dir = './input/train/audio/'
tr_sub_dirs = ['_background_noise_','bed','cat','up','down','right','eight','five']
#tr_sub_dirs = '_background_noise_ bed bird cat dog down eight five four go happy house left marvin nine no off on one right seven sheila six stop three tree two up wow yes zero'.split()
#tr_sub_dirs = ['_background_noise_','bed','bird']
file_ext = '*.wav'
for l, sub_dir in enumerate(tr_sub_dirs):
    print(os.path.join(parent_dir, sub_dir, file_ext))

./input/train/audio/_background_noise_/*.wav
./input/train/audio/bed/*.wav
./input/train/audio/cat/*.wav
./input/train/audio/up/*.wav
./input/train/audio/down/*.wav
./input/train/audio/right/*.wav
./input/train/audio/eight/*.wav
./input/train/audio/five/*.wav


In [194]:
L = 16000
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()
#legal_labels = 'bed bird cat dog down eight five four go happy house left marvin nine no off on one right seven sheila six stop three tree two up wow yes zero silence unknown'.split()

#src folders
root_path = r'.'
out_path = r'.'
model_path = r'.'
train_data_path = os.path.join(root_path, 'input', 'train', 'audio')
test_data_path = os.path.join(root_path, 'input', 'test', 'audio')

#train_data_path = os.path.join(root_path, 'input', 'train_test')
#test_data_path = os.path.join(root_path, 'input', 'test', 'audio')

In [195]:
def list_wavs_fname(dirpath, ext='wav'):
    #print(dirpath)
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

In [196]:
def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

In [197]:
labels, fnames = list_wavs_fname(train_data_path)

In [198]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

def extract_features(parent_dir,sub_dirs,file_ext="*.wav",bands = 60, frames = 41):
    start_time = time.time();
    window_size = 512 * (frames - 1)
    log_specgrams = []
    labels = []
    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            sound_clip,s = librosa.load(fn)
            #print(fn.split('/'))
            #print(fn.split('/')[-2].split('-')[-1])
            
            label = fn.split('/')[-2].split('-')[-1]
            for (start,end) in windows(sound_clip,window_size):
                #print(int(start) , int(end))
                if(len(sound_clip[int(start):int(end)]) == window_size):
                    signal = sound_clip[int(start):int(end)]
                    melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                    logspec = librosa.logamplitude(melspec)
                    logspec = logspec.T.flatten()[:, np.newaxis].T
                    log_specgrams.append(logspec)
                    labels.append(label)
                    #print(label)
                    #return
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
        #print(np.array(features).shape)
    #print(labels)
    print("--- %s seconds ---" % str(datetime.timedelta(seconds=(time.time() - start_time))))
    return np.array(features), np.array(labels)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [199]:
import glob

In [200]:
#tr_features,tr_labels = extract_features(parent_dir,tr_sub_dirs)
#print(tr_features.shape, tr_labels.shape)

In [ ]:
tr_features,tr_labels = extract_features(parent_dir,tr_sub_dirs)

In [166]:
print(tr_features.shape, tr_labels.shape )
tr_labels

(8416, 60, 41, 2) (8416,)


array(['_background_noise_', '_background_noise_', '_background_noise_',
       ..., 'down', 'down', 'down'],
      dtype='<U18')

In [167]:
#x_train = np.array(x_train)
#x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
y_train = label_transform(tr_labels)
label_index = y_train.columns.values
y_train = y_train.values
y_train = np.array(y_train)
del labels, fnames
gc.collect()

x_train = tr_features

In [168]:
y_train

array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       ..., 
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]], dtype=uint8)

In [169]:
print(x_train.shape, y_train.shape)

(8416, 60, 41, 2) (8416, 4)


In [170]:
len(y_train[0])

4

In [171]:
n1 = 60
n2 = 41
n3 = 2
nclass = len(np.array(legal_labels))-1
input_shape = (n1,n2,1)
nclass = len(y_train[0])

In [172]:
#original
#model-1
print('start')
input_shape  = (n1,n2,n3)

#nclass = len(np.array(legal_labels))-1
#nclass = nclass
start_time = time.time();
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(norm_inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)

img_1  = ZeroPadding2D((1,1))(img_1)

img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)

img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu,data_format='channels_first')(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)

img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(img_1))
dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

model = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

model.compile(optimizer=opt, loss=losses.binary_crossentropy)
model.summary()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=2017)
#vgg
batch_size = 128
nb_epoch = 1

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(x_train,y_train,nb_epoch= nb_epoch,batch_size = batch_size , validation_split=0.1)
model.fit(x_train, y_train, batch_size=128, validation_data=(x_valid, y_valid), epochs=5, shuffle=True, verbose=1)
model.save(os.path.join(model_path, 'lightCnnGPU_MFCC.model'))
print("finish")

start
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 60, 41, 2)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 60, 41, 2)         8         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 59, 40, 8)         72        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 58, 39, 8)         264       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 29, 19, 8)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 29, 19, 8)         0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 31, 21, 8)         0         
____

# model prediction
이제 모델을 완성했으니 y_test 에서 값을 가져와서 모델을  prediction하는 모델을 만들어 보자 

In [173]:
from keras.models import load_model
model = load_model("./lightCnnGPU_MFCC.model")

In [174]:
x_demo = x_train

In [175]:
x_demo.shape

(7574, 60, 41, 2)

In [176]:
x_train[0].shape
x_demo = np.expand_dims(x_train[0],axis=0)
x_demo.shape

(1, 60, 41, 2)

In [177]:
model.predict(x_demo)

array([[ 0.0033107 ,  0.01119525,  0.13847438,  0.84701973]], dtype=float32)

In [178]:
test_dir = './input/test/audio_test/'

In [179]:
#WITHOUT BATCH 
def extract_features_test(parent_dir,sub_dirs,file_ext="*.wav",bands = 60, frames = 41):
    start_time = time.time();
    window_size = 512 * (frames - 1)
    log_specgrams = []
    fnames = []

  
    for fn in glob.glob(os.path.join(test_dir, file_ext)):
        #print(fn.split('/')[-1])
        sound_clip,s = librosa.load(fn)
       
        for (start,end) in windows(sound_clip,window_size):
            #print(int(start) , int(end))
            if(len(sound_clip[int(start):int(end)]) == window_size):
                signal = sound_clip[int(start):int(end)]
                melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                logspec = librosa.logamplitude(melspec)
                logspec = logspec.T.flatten()[:, np.newaxis].T
                log_specgrams.append(logspec)
                fnames.append(fn.split('/')[-1])
                #labels.append(label)
                #print(label)
                #return
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
        #print(np.array(features).shape)
    #print(labels)
    print("--- %s seconds ---" % str(datetime.timedelta(seconds=(time.time() - start_time))))
    return np.array(features), np.array(fnames)


In [180]:
#BATCH TRYING EXAMPLE 
def extract_features_test(test_dir,sub_dirs,file_ext="*.wav",bands = 60, frames = 41,batch=16):
    start_time = time.time();
    i=0
    window_size = 512 * (frames - 1)
    log_specgrams = []
    labels = []
    
    for fn in glob.glob(os.path.join(test_dir, file_ext)):
        
        sound_clip,s = librosa.load(fn)
        if i==0:
            log_specgrams = []
            labels = []
        for (start,end) in windows(sound_clip,window_size):
            
            if(len(sound_clip[int(start):int(end)]) == window_size):
                signal = sound_clip[int(start):int(end)]
                melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                logspec = librosa.logamplitude(melspec)
                logspec = logspec.T.flatten()[:, np.newaxis].T
                log_specgrams.append(logspec)
                
                if i==batch:
                    i=0
                    logspec = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
                    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
                    for i in range(len(features)):
                        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
                    print('feature->', np.array(features))
                    yield np.array(features), np.array(labels)
        if i < batch:
                logspec = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
                features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
                for i in range(len(features)):
                    features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
                print('feature->', np.array(features))
                yield np.array(features), np.array(labels)
   
    print("--- %s seconds ---" % str(datetime.timedelta(seconds=(time.time() - start_time))))
    return np.array(features), np.array(labels)

In [181]:
#WITH BATCH 
def extract_features_test(parent_dir,sub_dirs,file_ext="*.wav",bands = 60, frames = 41,batch = 3):
    start_time = time.time();
    window_size = 512 * (frames - 1)
    log_specgrams = []
    fnames = []
    
    k = 0
    print('os.path->',os.path.join(test_dir, file_ext))
  
    for fn in glob.glob(os.path.join(test_dir, file_ext)):
        print('fn->', fn)
        sound_clip,s = librosa.load(fn)
        if k==0:
            print('i==0')
            log_specgrams = []
            fnames = []
            k = k+1
        else:
            print('i==else')
            k = k+1
 
            for (start,end) in windows(sound_clip,window_size):
                #print(int(start) , int(end))
                if(len(sound_clip[int(start):int(end)]) == window_size):
                    signal = sound_clip[int(start):int(end)]
                    melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                    logspec = librosa.logamplitude(melspec)
                    logspec = logspec.T.flatten()[:, np.newaxis].T
                    log_specgrams.append(logspec)
                    fnames.append(fn.split('/')[-1])

            if k==batch:
                print('i==batch')
                k = 0
                log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
                features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
                for i in range(len(features)):
                    features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
                print('yielding...',i)
                yield np.array(features), np.array(fnames)
        
        
    print("--- %s seconds ---" % str(datetime.timedelta(seconds=(time.time() - start_time))))
       

In [182]:
#import glob
#tr_features_test,tr_labels_test = extract_features_test(test_dir,tr_sub_dirs,batch=16)
#print(tr_features.shape, tr_labels_test.shape)

In [183]:
#tr_labels_test[0]

In [184]:
#tr_features_test[0].shape
#trying = np.expand_dims(tr_features[0], axis=0)
#model.predict(trying)

In [185]:
#extract feature test
import glob
new_sample_rate = 8000
print('start')
start_time = time.time();
index = []
results = []
for  imgs,fnames in extract_features_test(test_dir,sub_dir):
    print(imgs.shape)
    predicts = model.predict(imgs)
    predicts = np.argmax(predicts, axis=1)
    predicts = [label_index[p] for p in predicts]
    index.extend(fnames)
    results.extend(predicts)

index = [w.replace('./input/test/audio_test/', '') for w in index]
df = pd.DataFrame(columns=['fname', 'label'])
df['fname'] = index
df['label'] = results

df.to_csv(os.path.join(out_path, 'sub_300.csv'), index=False)
print("--- %s seconds ---" % str(datetime.timedelta(seconds=(time.time() - start_time))))

start
os.path-> ./input/test/audio_test/*.wav
fn-> ./input/test/audio_test/clip_0a1d41d64.wav
i==0
fn-> ./input/test/audio_test/clip_0a0a9fa8e.wav
i==else
fn-> ./input/test/audio_test/clip_0a2c1f0e1.wav
i==else
i==batch
yielding... 1
(2, 60, 41, 2)
fn-> ./input/test/audio_test/clip_0a1c321ee.wav
i==0
fn-> ./input/test/audio_test/clip_0a1daeb27.wav
i==else
fn-> ./input/test/audio_test/clip_0a0ba79ac.wav
i==else
i==batch
yielding... 1
(2, 60, 41, 2)
fn-> ./input/test/audio_test/clip_0a0e5bbcd.wav
i==0
fn-> ./input/test/audio_test/clip_0a2b4d369.wav
i==else
fn-> ./input/test/audio_test/clip_0a1da4f17.wav
i==else
i==batch
yielding... 1
(2, 60, 41, 2)
fn-> ./input/test/audio_test/clip_0a0adcba6.wav
i==0
fn-> ./input/test/audio_test/clip_0a0b35e75.wav
i==else
fn-> ./input/test/audio_test/clip_00a2c3e3d.wav
i==else
i==batch
yielding... 1
(2, 60, 41, 2)
fn-> ./input/test/audio_test/clip_0a0f79193.wav
i==0
fn-> ./input/test/audio_test/clip_0a0fc880e.wav
i==else
fn-> ./input/test/audio_test/clip